# Datamodels module examples

In [1]:
from hero_db_utils.datamodels import datamodel
from hero_db_utils.datamodels.fields import ForeignKeyField, AutoSerialField
import pandas as pd
import dotenv
dotenv.load_dotenv(override=True);

@datamodel
class Store:
    address_id:ForeignKeyField("address","address_id")
    last_update:pd.to_datetime
    manager_staff_id:ForeignKeyField("staff", "staff_id") = None
    store_id:AutoSerialField() = None

@datamodel
class Address:
    address:str
    address2:str
    district:str
    city_id:ForeignKeyField("city", "city_id")
    postal_code:str
    phone:str
    last_update:pd.to_datetime
    address_id:AutoSerialField() = None

@datamodel
class City:
    city:str
    country_id:ForeignKeyField("country", "country_id")
    last_update:str
    city_id:AutoSerialField() = None

@datamodel
class Country:
    country:str
    last_update:pd.to_datetime
    country_id:AutoSerialField() = None

@datamodel
class Staff:
    first_name:str
    last_name:str
    address_id:ForeignKeyField("address","address_id")
    email:str
    store_id:ForeignKeyField("stores", "store_id")
    active:bool
    username:str
    password:str
    last_update:pd.to_datetime
    staff_id:AutoSerialField() = None


True

In [3]:
%env HERO_POSTGRES_DBNAME=dvd_rentals

env: HERO_POSTGRES_DBNAME=dvd_rentals


Select all from a model in the database:

In [3]:
Store.objects.all()

,store_id,manager_staff_id,address_id,last_update
0,1,1,1,2022-03-13 20:17:25.832507
1,2,2,2,2022-03-15 21:31:54.117213


Filter from the table:

In [4]:
from hero_db_utils.queries.postgres import QueryOp

Address.objects.filter(address_id=QueryOp.value_in([1,2]))

,address_id,address,address2,district,city_id,postal_code,phone,last_update
0,1,47 MySakila Drive,None,Alberta,300,,,2006-02-15 09:45:30
1,2,28 MySQL Boulevard,None,QLD,576,,,2006-02-15 09:45:30


Join on related table for column "address_id":

In [5]:
Store.objects.fetch_related("address_id")

,store_id,manager_staff_id,address_id,last_update,address,address2,district,city_id,postal_code,phone,last_update
0,1,1,1,2022-03-13 20:17:25.832507,47 MySakila Drive,None,Alberta,300,,,2006-02-15 09:45:30
1,2,2,2,2022-03-15 21:31:54.117213,28 MySQL Boulevard,None,QLD,576,,,2006-02-15 09:45:30


Retrieve an object from the database:

In [6]:
store = Store.objects.get(store_id=2)
store.data

address_id                                   2
last_update         2022-03-15 21:31:54.117213
manager_staff_id                             2
store_id                                     2
dtype: object

Update a model in the database:

In [19]:
store.update(address_id=3)

In [20]:
store.address_id

3

In [21]:
Store.objects.all()

,store_id,manager_staff_id,address_id,last_update
0,1,1,1,2022-03-13 20:17:25.832507
1,2,2,3,2022-03-15 21:42:54.841072


Create a new staff member:

In [10]:
Staff.objects.all()

,staff_id,first_name,last_name,address_id,email,store_id,active,username,password,last_update,picture
0,1,Mike,Hillyer,3,Mike.Hillyer@sakilastaff.com,1,True,Mike,8cb2237d0679ca88db6464eac60da96345513964,2006-05-16 16:13:11.793280,"[b'\x89', b'P', b'N', b'G', b'\r', b'\n', b'Z'..."
1,2,Jon,Stephens,4,Jon.Stephens@sakilastaff.com,2,True,Jon,8cb2237d0679ca88db6464eac60da96345513964,2006-05-16 16:13:11.793280,None


In [15]:
new_staff = Staff(
    first_name="Harry",
    last_name="Potter",
    active = True,
    store_id=1,
    address_id=2,
    email="Harry.Potter@hogwarts.com",
    username="theboywholived",
    password="alohomora",
    last_update="now"
)
new_staff.data

first_name                          Harry
last_name                          Potter
address_id                              2
email           Harry.Potter@hogwarts.com
store_id                                1
active                               True
username                   theboywholived
password                        alohomora
last_update    2022-03-15 21:38:00.341756
staff_id                             None
dtype: object

In [16]:
# Insert staff in the database:
new_staff.insert()

In [30]:
# Check it exists in the database:
Staff.objects.all()

,staff_id,first_name,last_name,address_id,email,store_id,active,username,password,last_update,picture
0,1,Mike,Hillyer,3,Mike.Hillyer@sakilastaff.com,1,True,Mike,8cb2237d0679ca88db6464eac60da96345513964,2006-05-16 16:13:11.793280,"[b'\x89', b'P', b'N', b'G', b'\r', b'\n', b'Z'..."
1,2,Jon,Stephens,4,Jon.Stephens@sakilastaff.com,2,True,Jon,8cb2237d0679ca88db6464eac60da96345513964,2006-05-16 16:13:11.793280,None
2,3,Harry,Potter,2,Harry.Potter@hogwarts.com,1,True,theboywholived,alohomora,2022-03-15 21:38:00.341756,None
